<a href="https://colab.research.google.com/github/TonioHathos/AI_Valeo/blob/main/code_Antoine/Mod%C3%A8le_NN_Antoine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import classification_report

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import random



!pip install category_encoders
import category_encoders as ce




# The following library is to plot the loss during training
# https://github.com/stared/livelossplot
! pip install livelossplot
from livelossplot import PlotLossesKerasTF


# Import the visualization library I prepared for you
! wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/visualization.py
from visualization import plot_conf_mat


# The following is to be able to mount Google Drive
from google.colab import drive

import os
from os.path import isfile

from functools import partial

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-05-24 20:49:57--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/visualization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13023 (13K) [text/plain]
Saving to: ‘visualization.py.2’

visualization.py.2  100%[===================>]  12.72K  --.-KB/s    in 0s      

2023-05-24 20:49:58 (27.0 MB/s) - ‘visualization.py.2’ saved [13023/13023]



### Mount your Google Drive

Training a neural network may take long time and you don't want to do it every time. Once you've trained a model, it is better to save it, so that you can use it immediately for prediction next times.

Unfortunately, the storage on Google is reset every time. You need to store all persistent data in you Google Drive.

You need therefore to mount your Google Drive, which you will use later in this notebook

In [28]:
mount_point = '/content/gdrive' # Always the same, don't change it
drive.mount(mount_point, force_remount=True)
drive_path = mount_point + '/My Drive/' # Always the same, don't change it

# Replace the following folder with some folder inside your google drive
my_path = drive_path + \
  'TSP/Apprentissage_automatique_réseau/'

Mounted at /content/gdrive


## Récupérer dataset modifié

In [29]:
! rm -f *.csv
! wget https://raw.githubusercontent.com/TonioHathos/AI_Valeo/master/df_training_mod.csv
df_training_mod = pd.read_csv('df_training_mod.csv')

--2023-05-24 20:40:34--  https://raw.githubusercontent.com/TonioHathos/AI_Valeo/master/df_training_mod.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2666373 (2.5M) [text/plain]
Saving to: ‘df_training_mod.csv’

df_training_mod.csv 100%[===================>]   2.54M  --.-KB/s    in 0.08s   

2023-05-24 20:40:34 (31.9 MB/s) - ‘df_training_mod.csv’ saved [2666373/2666373]



In [30]:
df_training_mod.columns

Index(['Unnamed: 0', 'OP070_V_1_angle_value', 'OP090_SnapRingPeakForce_value',
       'OP070_V_2_angle_value', 'OP120_Rodage_I_mesure_value',
       'OP090_SnapRingFinalStroke_value', 'OP110_Vissage_M8_torque_value',
       'OP100_Capuchon_insertion_mesure', 'OP120_Rodage_U_mesure_value',
       'OP090_StartLinePeakForce_value', 'OP110_Vissage_M8_angle_value',
       'OP090_SnapRingMidPointForce_val', 'OP070_V_2_torque_value',
       'Binar OP130_Resultat_Global_v'],
      dtype='object')

In [31]:
df_training_mod.drop(columns=['Unnamed: 0'])

,OP070_V_1_angle_value,OP090_SnapRingPeakForce_value,OP070_V_2_angle_value,OP120_Rodage_I_mesure_value,OP090_SnapRingFinalStroke_value,OP110_Vissage_M8_torque_value,OP100_Capuchon_insertion_mesure,OP120_Rodage_U_mesure_value,OP090_StartLinePeakForce_value,OP110_Vissage_M8_angle_value,OP090_SnapRingMidPointForce_val,OP070_V_2_torque_value,Binar OP130_Resultat_Global_v
0,180.4,190.51,173.1,113.64,12.04,12.16,0.0,11.97,26.37,18.8,109.62,6.60,0
1,138.7,147.70,163.5,109.77,12.12,12.19,1.0,11.97,21.03,18.5,105.48,6.40,0
2,180.9,150.87,181.2,109.79,11.86,12.24,0.0,11.97,25.81,17.5,100.03,6.61,0
3,173.5,159.56,151.8,113.25,11.82,12.35,1.0,11.97,24.62,15.6,104.94,6.61,0
4,174.5,172.29,177.5,112.88,12.07,12.19,0.0,11.97,29.22,33.6,99.19,6.61,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34510,158.2,168.40,180.6,110.65,11.88,12.23,0.0,11.98,24.03,14.8,98.76,6.61,0
34511,139.4,146.24,154.5,108.14,12.04,12.27,1.0,11.97,21.52,15.5,98.37,6.40,0
34512,165.5,157.34,149.8,112.77,12.09,12.17,0.0,11.97,22.65,24.4,92.92,6.62,0
34513,152.4,165.07,140.0,111.19,12.06,12.34,0.0,11.97,20.77,13.6,101.21,6.60,0


### Class imbalance

In [37]:
labels_raw = df_training_mod['Binar OP130_Resultat_Global_v']
labels_raw

0        0
1        0
2        0
3        0
4        0
        ..
34510    0
34511    0
34512    0
34513    0
34514    0
Name: Binar OP130_Resultat_Global_v, Length: 34515, dtype: int64

In [33]:
Counter(labels_raw)

Counter({0: 34210, 1: 305})

On va corriger ce déséquilibre en applicant la fonction SMOTE()

In [35]:
smote = SMOTE()
df_training, labels = smote.fit_resample(df_training_mod, labels_raw)

In [36]:
Counter(labels)

Counter({0: 34210, 1: 34210})

### Scaling

In [40]:
scaler = StandardScaler()
scaler.fit(df_training)

df_training = scaler.transform(df_training)

In [44]:
print('X_train scaled\n', df_training[0:6, :] )

X_train scaled
 [[-1.73929397  1.53209786  3.31075928  1.02963778  0.06104866  0.4056304
  -1.66552284 -1.20963666 -0.32308274  1.12992514  0.12346919  1.89205798
   0.763782   -1.        ]
 [-1.73919268 -1.42729323 -0.84397194  0.32582806 -1.17037696  0.99042589
  -1.14542288  0.9792669  -0.32308274 -1.18456635  0.07563956  1.20261752
  -1.4134327  -1.        ]
 [-1.73909138  1.56758216 -0.53632191  1.62347722 -1.16401301 -0.91015946
  -0.27858961 -1.20963666 -0.32308274  0.88720693 -0.08379254  0.29502077
   0.87264273 -1.        ]
 [-1.73899009  1.04241444  0.3070468  -0.53194003 -0.06304849 -1.2025572
   1.62844359  0.9792669  -0.32308274  0.37143074 -0.38671354  1.1126905
   0.87264273 -1.        ]
 [-1.73888879  1.11338305  1.54249946  1.35221723 -0.18078169  0.62492871
  -1.14542288 -1.20963666 -0.32308274  2.36518746  2.48306431  0.1551343
   0.87264273 -1.        ]
 [-1.73878749 -0.00792101  0.91167131  1.41819939 -0.43215798 -0.76396058
  -1.83888949 -1.20963666 -0.32308274  

In [45]:
labels

0        0
1        0
2        0
3        0
4        0
        ..
68415    1
68416    1
68417    1
68418    1
68419    1
Name: Binar OP130_Resultat_Global_v, Length: 68420, dtype: int64

In [46]:
df_training['Binar OP130_Resultat_Global_v']

IndexError: ignored